# Gaussian process for geographic data (GP lab part 2)

**Machine Learning, University of Zaragoza, Ruben Martinez-Cantin**

In this lab, you will implement a Gaussian process for air quality data, obtained from the City Hall in their [open data site](https://www.zaragoza.es/sede/portal/datos-abiertos/servicio/catalogo/131).

  *adapted from a lab by Luis Montesano*

In [ ]:
#@title install libraries
!pip install GPy

In [ ]:
#@title import libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from geopy import Nominatim, Photon
from PIL import Image
from pyproj import Transformer
#from sklearn.gaussian_process import GaussianProcessRegressor
#from sklearn.gaussian_process.kernels import RBF, WhiteKernel
import GPy

# descomenta %matplotlib qt si prefieres plots interactivos
%matplotlib inline
# %matplotlib qt

## 0. Setup y Auxiliary Funtions

### 0.1 Coordinate transformation

The geographical coordinates we will use are given in degrees (*latitude* and *longitude*). Therefore, we need a way to transform them (or *project* them) to metric units (e.g. meters), as these are more suitable in the calculation of distances needed in **Gaussian processes**.

For these transformations, we will use the **PyProj** library. [ [code](https://github.com/pyproj4/pyproj) | [docs](https://pyproj4.github.io/pyproj/stable/) ].


&#9432; **EXTRA INFO**

These transformations are performed using the following **coordinate reference systems (CRS)**:
* EPSG:4326 (WGS84) as being the appropriate CRS for latitude and longitude coordinates (represented in degrees) [<a href="https://epsg.io/?q=4326">link</a>].
* EPSG:32630 (UTM, zona 30) as this CRS is used to express the coordinates in meters, projecting them to a local 2D plane. Zone 30 is the one corresponding to Zaragoza [<a href="https://www.ign.es/web/coordenadas-de-stations-ergnss">link</a>].


In [ ]:
transformer_longlat_xy = Transformer.from_crs(4326, 32630, always_xy=True)


def from_longlat_to_xy(long, lat):
    """Transform longitude and latitude coordinates to x, y coordinates in
    CRS 32630 (UTM, zone 30)

    Args:
        long: (n,) array or float with longitude (degrees)
        lat: (n,) array or float with latitude (degrees)

    Returns:
        x: (n,) array or float with x coordinates in CRS 32630
        y: (n,) array or float with y coordinates in CRS 32630
    """
    return transformer_longlat_xy.transform(long, lat)


def from_xy_to_longlat(x, y):
    """Transform x, y coordinates in CRS 32630 (UTM, zone 30) to longitude and
    latitude coordinates

    Args:
        x: (n,) array or float with x coordinates in CRS 32630
        y: (n,) array or float with y coordinates in CRS 32630

    Returns:
        long: (n,) array or float with longitude (degrees)
        lat: (n,) array or float with latitude (degrees)
    """
    return transformer_longlat_xy.transform(x, y, direction="inverse")

### 0.2 Generation and data loading


First, we need to obtain the **geographic coordinates** (latitude and longitude) of the air quality measurement **stations** in Zaragoza.

Knowing the station location (street or address) for each station, we use `Photon` *geodecoder* to obtain an automated (albeit approximate) location of the stations using `OpenStreetMap` data. For that, we use the `Geopy` library [[*GH*](https://github.com/geopy/geopy) | [*Docs*](https://geopy.readthedocs.io/en/stable/)].

The address of each station can be found in the city hall [website](https://www.zaragoza.es/sede/portal/medioambiente/calidad-aire/). We store the data in a `Pandas` dataframe to simplify access and to show the results in a table form.

In [ ]:
stations = {
    "Actur": "Calle Cineasta Carlos Saura, 50018",
    "El Picarral": "Av. San Juan de la Peña / C.S. Picarral, 50015",
    "Roger de Flor": "Calle Roger de Flor, 50017",
    "Jaime Ferran": "Calle Jaime Ferrán, 50014",
    "Renovales": "Paseo de Mariano Renovales, 50006",
    "Las Fuentes": "Calle María de Aragón, 50013",
    "Centro": "Calle José Luis Albareda, 50004",
    "Avda. Soria": "Avenida Ciudad de Soria, 50010, Zaragoza, España",
}

latitudes, longitudes = [], []
geolocator = Photon(user_agent="myGeocoder",timeout=100)
for name, address in stations.items():
    location = geolocator.geocode(address)
    latitudes.append(location.latitude)  # type: ignore
    longitudes.append(location.longitude)  # type: ignore

data = {
    "station": list(stations.keys()),
    "latitude": latitudes,
    "longitude": longitudes,
}
df_stations = pd.DataFrame(data)

df_stations

,station,latitude,longitude
0,Actur,41.675101,-0.888726
1,El Picarral,41.667799,-0.871634
2,Roger de Flor,41.650049,-0.917384
3,Jaime Ferran,41.665323,-0.862919
4,Renovales,41.633303,-0.891697
5,Las Fuentes,41.641563,-0.862971
6,Centro,41.649634,-0.885469
7,Avda. Soria,41.659058,-0.907933


Now, let's add the **metric coordinates**  $x, y$ (meters) corresponding to the *CRS UTM (zone 30)* of each station:

In [ ]:
df_stations[["x", "y"]] = df_stations.apply(
    lambda row: from_longlat_to_xy(row.longitude, row.latitude),
    axis=1,
    result_type="expand",
)
df_stations

,station,latitude,longitude,x,y
0,Actur,41.675101,-0.888726,675741.966418,4.615857e+06
1,El Picarral,41.667799,-0.871634,677184.810141,4.615082e+06
2,Roger de Flor,41.650049,-0.917384,673423.593261,4.613018e+06
3,Jaime Ferran,41.665323,-0.862919,677917.177571,4.614825e+06
4,Renovales,41.633303,-0.891697,675608.180010,4.611211e+06
5,Las Fuentes,41.641563,-0.862971,677978.316702,4.612187e+06
6,Centro,41.649634,-0.885469,676082.517258,4.613037e+06
7,Avda. Soria,41.659058,-0.907933,674186.377457,4.614037e+06


Last, we load the file `calidad_aire_2023.csv` corresponding to the air quality measured as the *concentration* [$\mu g/m^3$] of several chemical contaminants, measured in each station through 2023.

If you are working locally in your system or you already have downloaded the file, you can skip this step.

In [ ]:
#@title download data
!wget https://raw.githubusercontent.com/rmcantin/gp_lab/main/datasets/calidad_aire_2023.csv

--2024-10-28 14:26:08--  https://raw.githubusercontent.com/rmcantin/gp_lab/main/datasets/calidad_aire_2023.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 553388 (540K) [text/plain]
Saving to: ‘calidad_aire_2023.csv.1’

calidad_aire_2023.c 100%[===================>] 540.42K  --.-KB/s    in 0.005s  

2024-10-28 14:26:09 (115 MB/s) - ‘calidad_aire_2023.csv.1’ saved [553388/553388]



In [ ]:
#@title load downloaded data
path_calidad = "calidad_aire_2023.csv"
df_air_q = pd.read_csv(path_calidad, sep=",")
df_air_q

,id,title,contaminante,nivel,value,fecha,year
0,220230224nox,Roger de Flor,nox,NaN,20.1,24-02-2023,2023
1,720230224no2,Jaime Ferran,no2,Buena,27.5,24-02-2023,2023
2,720230224sh2,Jaime Ferran,sh2,Buena,1.7,24-02-2023,2023
3,820230224so2,Renovales,so2,Buena,9.6,24-02-2023,2023
4,820230224co,Renovales,co,Buena,0.2,24-02-2023,2023
...,...,...,...,...,...,...,...
9995,820230903nox,Renovales,nox,NaN,6.2,03-09-2023,2023
9996,1020230903co,Centro,co,Buena,0.2,03-09-2023,2023
9997,1220230903no2,Actur,no2,Buena,10.3,03-09-2023,2023
9998,1120230905co,Avda. Soria,co,Buena,0.2,05-09-2023,2023


### 0.3 Interactive visualization

In the next code, we develop the class `MapPlotter`, based in [**Plotly**](https://plotly.com/graphing-libraries/) (using the [map visualization API](https://plotly.com/python/mapbox-layers/)) which defines auxiliar methods to visualize the lab session results.

In [ ]:
class MapPlotter:
    """Interactive map visualization based on Plotly"""

    CENTER = {"lat": 41.65573, "lon": -0.88616}

    def __init__(self, zoom=12, center=None, showlegend=False, **kwargs):
        self.fig = go.Figure()
        self.fig.update_layout(
            mapbox_style="open-street-map",  # or "carto-positron",
            mapbox_zoom=zoom,
            mapbox_center=center or self.CENTER,
            margin={"r": 0, "t": 10, "l": 0, "b": 10},
            showlegend=showlegend,
            # autosize=True,
            **kwargs,
        )

    def add_longlat_points(
        self,
        lon,
        lat,
        s=15,
        color=None,
        text=None,
        edgecolor=None,
        name=None,
        adjust_center=True,
        **kwargs,
    ):
        if edgecolor is not None:
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=lat,
                    lon=lon,
                    mode="markers",
                    marker_size=1.1 * s,
                    marker_color=edgecolor,
                    showlegend=False,
                    **kwargs,
                )
            )

        self.fig.add_trace(
            go.Scattermapbox(
                lat=lat,
                lon=lon,
                mode="markers",
                marker_size=s,
                marker_color=color,
                text=text,
                name=name,
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_density_heatmap(self, lon, lat, z, radius=10, adjust_center=True, **kwargs):
        self.fig.add_trace(
            go.Densitymapbox(
                lat=lat.to_numpy(),
                lon=lon.to_numpy(),
                z=z,
                radius=radius,
                colorbar_title="μg/m³",
                **kwargs,
            )
        )

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

    def add_raster_heatmap(
        self,
        xgrid,
        ygrid,
        heatmap,
        cmap="Spectral_r",
        adjust_center=True,
        add_cbar=True,
        **kwargs,
    ):
        # rasterize the heatmap
        heatmap_ = (heatmap - heatmap.min()) / np.ptp(heatmap)
        heatmap_ = plt.get_cmap(cmap)(heatmap_)[..., :3]
        raster_hm = Image.fromarray((heatmap_ * 255).clip(0, 255).astype(np.uint8))

        # heatmap bounding box
        bbox = np.array(
            [
                [xgrid[0, 0], ygrid[0, 0]],  # top-left
                [xgrid[0, -1], ygrid[0, -1]],  # top-right
                [xgrid[-1, -1], ygrid[-1, -1]],  # bottom-right
                [xgrid[-1, 0], ygrid[-1, 0]],  # bottom-left
            ]
        )
        lon, lat = from_xy_to_longlat(bbox[:, 0], bbox[:, 1])
        coords = np.stack((lon, lat), axis=1)

        layers = [*self.fig.layout.mapbox.layers]  # type: ignore
        layers.append(
            {
                "sourcetype": "image",
                "source": raster_hm,
                "coordinates": coords,
                "below": "traces",
                **kwargs,
            }
        )
        self.fig.layout.mapbox.layers = layers  # type: ignore

        if adjust_center:
            self.adjust_center(lon.mean(), lat.mean())

        if add_cbar:
            # add colorbar via invisible scatter trace
            self.fig.add_trace(
                go.Scattermapbox(
                    lat=[None],
                    lon=[None],
                    mode="markers",
                    marker=dict(
                        colorscale=cmap,
                        color=heatmap,
                        cmin=heatmap.min(),
                        cmax=heatmap.max(),
                        showscale=True,
                        colorbar=dict(
                            title="μg/m³",
                            # titleside="right",
                            ticks="outside",
                        ),
                    ),
                )
            )

    def adjust_center(self, lon, lat):
        self.fig.update_layout(mapbox_center={"lat": lat, "lon": lon})

    def show(self):
        self.fig.show()

## 1. Gaussian process for geographical data

In this part of the lab, you must implement the **Gaussian process model** to interpolate the air quaility in Zaragoza and visualize the results obtained.

For this, you need to complete the following functions:

&#9432; **WARNING**

Some stations do not have concentration data for all the contaminants, because they lack the corresponding sensor. We recommend you to use the following contaminants which are available in all the stations:

* `nox`
* `no2`
* `no`
* `o3`

In [ ]:
#@title Utils to process the data
def create_regular_grid(X, npts=100):
    """Coordinate mesh grid of where to predict the air quaility using GPs

    Args:
        X: (nobs, 2) array of the xy coordinates of the observations
        npts: number of points in width and height in the grid.

    Returns:
        xgrid: (npts, npts) array with x coordinates of the grid
        ygrid: (npts, npts) array with y coordinates of the grid
    """
    # grid limits
    x_min, x_max = X[..., 0].min(), X[..., 0].max()
    y_min, y_max = X[..., 1].min(), X[..., 1].max()

    # we add 10% padding/margin
    padding_x = (x_max - x_min) * 0.1
    padding_y = (y_max - y_min) * 0.1

    x_min -= padding_x
    x_max += padding_x
    y_min -= padding_y
    y_max += padding_y

    # regular grid
    x, y = np.linspace(x_min, x_max, npts), np.linspace(y_min, y_max, npts)
    xgrid, ygrid = np.meshgrid(x, y)
    return xgrid, ygrid

def load_air_quality_data(df_air_q, contaminant="o3", log_data=False):
    """Load air quality data for GP estimation in Zaragoza

    Args:
        df_air_q: Air quality DataFrame
        contaminant: str, contaminant name to estimate
        log_data: if True, print the observations of that contaminant in each
            station

    Returns:
        X: (n_stations, 2) array with the location of the stations
        measured_concentration: (n_stations, ) array with the observations for
            contaminant at the stations (average of 2023).
    """
    # first, we check that the contamint is in the DataFrame
    assert (
        contaminant in df_air_q["contaminante"].unique()
    ), "Contaminant not found"

    # we filter all data regarding the contamint
    data_contaminante = df_air_q[df_air_q["contaminante"] == contaminant]

    # As observation, we use the mean value along 2023 of each contamint in
    # each station
    observations = data_contaminante.groupby("title")["value"].mean()

    # We create an auxiliary dataframe where we add the observation to each station
    df_est = df_stations.copy()
    df_est["concen."] = df_est["station"].map(observations)
    assert (
        df_est["concen."].notnull().all()
    ), "Missing observations for some stations"

    if log_data:
        print(df_est)

    # Input data for the Gaussian process
    X = df_est[["x", "y"]].values    # location of the stations
    measured_concentrations = df_est["concen."].values

    return X, measured_concentrations

In [ ]:
#@title Function to predict concentration based on a Gaussian process
## @markdown Complete the TODO sections
from sklearn.metrics import mean_squared_error


def air_quality_prediction(X, measured_concentrations, gp_kernel, npts_grid=100):
    """GP-based air quality estimation in Zaragoza

    Args:
        X: (n_stations, 2) array with the location of the stations
        measured_concentration: (n_stations, ) array with the observations for
            contaminant at the stations (average of 2023).
        gp_kernel: Kernel for the GP to train
        npts_grid: int, number of points in width and height in the grid where
            we are going to predict the air quality

    Returns:
        (xgrid, ygrid): tupla with all x,y coordinates of the grid in
            shape (npts, npts)
        pred_concentration: (npts, npts) array with the predicted mean value of
            the contamint at each point of the grid.
        sigma_concentration: (npts, npts) array with the predicted standard deviation
            of the contamint at each point of the grid.
    """

    # Gaussian process training using the observations
    # TODO: Add the training code here using GPy and gp_kernel
    #
    # IMPORTANT:
    #    Sometimes Python vectors are represented with an empty dimension (n, ).
    #    Many libraries consider verctors as a special case of a 1D matrix.
    #    Thus, we need to make the vector (n, 1). For that, you can use either:
    #     - numpy.atleast_2d
    #     - numpy.newaxis
    #    Check the documentation for each method. Note that (1, n) is a different
    #    matrix, but you can then transpose the result.
    y_train = np.atleast_2d(measured_concentrations).T
    print(f"Shape of y_train: {y_train.shape}")

    model = GPy.models.GPRegression(X, y_train, gp_kernel)
    model.optimize(messages=True)

    # Create a regular mesh to predict the air quality in all the city.
    xgrid, ygrid = create_regular_grid(X, npts=npts_grid)
    X_pred = np.vstack([xgrid.ravel(), ygrid.ravel()]).T
    # Gaussian process prediction at the grid points
    # TODO: Using the trained model, predict the concentration mean and standar
    # deviation at the grid locations
    #
    # IMPORTANT:
    #     You might need to reshape the values of the grid. GPy expects a single
    #     matrix with size (n_predictions, input_dim). To reshape the grids you
    #     can use the following methods:
    #     -numpy.ravel
    #     -numpy.vstack
    #     and you might also need to transpose the results.
    pred_mean, pred_var = model.predict(X_pred)



    # Reshape the GP predictions to adjust it to the grid: shape (npts, npts)
    # This will be required for plotting. Right now we are returning random values
    # that you can use for checking the plots.
    #
    # TODO: replace the random values with the GP prediction.
    #
    # IMPORTANT:
    #      For reshape, you can direcly use numpy.reshape. Also, check that you
    #      return the standard deviation and not the variance.

    # Comment these two lines and make your own predictions
    # pred_concentration = np.random.rand(npts_grid, npts_grid)
    # std_concentration = np.random.rand(npts_grid, npts_grid)
    pred_concentration = pred_mean.reshape((npts_grid, npts_grid))
    std_concentration = np.sqrt(pred_var).reshape((npts_grid, npts_grid))

    log_likelihood = model.log_likelihood()

    return (xgrid, ygrid), pred_concentration, std_concentration, log_likelihood

## Process data and Kernel definition

Using GPy, you need to define the kernel. Try different kernels and combinations of them.

IMPORTANT: GPy uses gradient descent to learn the hyperparameters. This might fail if the hyperparameters are quite far from the optimal values as they get stuck in local minima. You can solve this in two ways:
- Either give an good initial value for the hyperparameters (order of magnitude). For example: as a rule of thumb, the lenghtscale should be in the same scale as the GP inputs and the variance should be in the same scale as the GP outputs.
- Alternatively, if we assume that the original data follows a Gaussian distribution $X \sim N(\mu_X, \sigma_X)$, then you can normalize your data such that the mean value is 0 and the variance is 1, that is, $newX \sim N(0,1)$. Then:
$$ newX = \frac{X - \mu_X}{\sigma_X}$$
If you train your GP with the normalize data, then, you need to remember to apply the same factor to the prediction location

$$ new\_prediction = \frac{original\_prediction - \mu_X}{\sigma_X}$$

**Data normalization** is always a good practice if done properly and improves the numerical performance in **all kind of problems**.

In [ ]:
kernel = GPy.kern.RBF(input_dim=2, variance=1.0, lengthscale=1.0) + GPy.kern.Linear(input_dim=2)

#Loading the data for the selected contaminant.
#TODO: Test different contaminants
X, y = load_air_quality_data(df_air_q, contaminant="no2")

#TODO: Normalize your data if needed
X_mean, X_std = X.mean(axis=0), X.std(axis=0)
y_mean, y_std = y.mean(), y.std()

X_norm = (X - X_mean) / X_std
y_norm = (y - y_mean) / y_std

# Compute the predictions of the Gaussian process
grid_norm, concentration_prediction_norm, concentration_std_norm, log_likelihood_norm = air_quality_prediction(X_norm, y_norm, kernel)

# Desnormalizar grid
xgrid_norm, ygrid_norm = grid_norm
xgrid = (xgrid_norm * X_std[0]) + X_mean[0]  # Desnormalizar la primera dimensión
ygrid = (ygrid_norm * X_std[1]) + X_mean[1]  # Desnormalizar la segunda dimensión
grid = (xgrid, ygrid)

#TODO: Make sure the grid is NOT affected by the normalization.
concentration_prediction = concentration_prediction_norm * y_std + y_mean
concentration_std = concentration_std_norm * y_std
log_likelihood = log_likelihood_norm * y_std**2

Shape of y_train: (8, 1)


In [ ]:
#TODO: You can call create_regular_grid again if needed
xgrid, ygrid = create_regular_grid(X)
plotter = MapPlotter()
plotter.add_density_heatmap(lon=df_stations["longitude"], lat=df_stations["latitude"], z=concentration_prediction)
plotter.show()

## Visualization

Let's plot the prediction and standard deviation in two separate maps.


In [ ]:
fig = MapPlotter()

# Adding the stations as points in the maps:
fig.add_longlat_points(
    df_stations["longitude"],
    df_stations["latitude"],
    text=df_stations.station,
    color="white",
    edgecolor="black",
)

# Adding the predicted concentration as a heatmap in the grid
fig.add_raster_heatmap(grid[0], grid[1], concentration_prediction_norm, opacity=0.6)

# Show map
fig.show()

In [ ]:
fig2 = MapPlotter()

# Adding the stations as points in the maps:
fig2.add_longlat_points(
    df_stations["longitude"],
    df_stations["latitude"],
    text=df_stations.station,
    color="white",
    edgecolor="black",
)

# Adding the predicted concentration as a heatmap in the grid
fig2.add_raster_heatmap(grid[0], grid[1], concentration_std_norm, opacity=0.6)

# Show map
fig2.show()

# Your tasks

- Use different kernels to predict the concentration for each of the contaminants. Justify your selection.

- Explain how did you solve the problem with the hyperparameter learning.

- If you can invest in a new station. Where would you place it?

## 1. Try different kernels

In [ ]:
def normalize_contaminants_data(df_air_q, contaminants):
    """
    Normalizes the concentration data for each contaminant.

    Args:
        df_air_q: DataFrame containing air quality data.
        contaminants: List of contaminant names to normalize (e.g., ["nox", "no2", "no", "o3"]).

    Returns:
        normalized_data: Dictionary containing normalized data, means, and standard deviations for each contaminant.
        Format: { contaminant: {'X_norm': X_norm, 'y_norm': y_norm, 'X_mean': X_mean, 'X_std': X_std, 'y_mean': y_mean, 'y_std': y_std} }
    """
    normalized_data = {}

    for contaminant in contaminants:
        X, y = load_air_quality_data(df_air_q, contaminant=contaminant)

        X_mean, X_std = X.mean(axis=0), X.std(axis=0)
        y_mean, y_std = y.mean(), y.std()

        X_norm = (X - X_mean) / X_std
        y_norm = (y - y_mean) / y_std

        normalized_data[contaminant] = {
            'X_norm': X_norm,
            'y_norm': y_norm,
            'X_mean': X_mean,
            'X_std' : X_std,
            'y_mean': y_mean,
            'y_std': y_std
        }

    return normalized_data


In [ ]:
import GPy
import numpy as np

def predict_with_different_kernels(df_air_q, contaminants, kernels, normalized_data, kfold=False):
    """
    Uses different kernels to predict the concentration for each contaminant.

    Args:
        df_air_q: DataFrame containing air quality data.
        contaminants: List of contaminants to predict.
        kernels: List of tuples (kernel name, kernel) to be tested.

    Returns:
        results: Dictionary with predictions, standard deviations, and MSE for each contaminant and kernel.
    """
    results = {}

    for contaminant in contaminants:
        print(f"\nProcessing contaminant: {contaminant}")

        data = normalized_data[contaminant]
        X_norm = data['X_norm']
        y_norm = data['y_norm']
        X_mean = data['X_mean']
        X_std = data['X_std']
        y_mean = data['y_mean']
        y_std = data['y_std']

        results[contaminant] = {}

        for kernel_name, kernel in kernels:
            print(f"  - Using kernel: {kernel_name}")
            grid_norm, concentration_prediction_norm, concentration_std_norm, log_likelihood_norm = air_quality_prediction(X_norm, y_norm, kernel)

            # Denormalize the grid coordinates
            xgrid_norm, ygrid_norm = grid_norm
            xgrid = (xgrid_norm * X_std[0]) + X_mean[0]
            ygrid = (ygrid_norm * X_std[1]) + X_mean[1]
            grid = (xgrid, ygrid)

            # Denormalize the predictions and uncertainty measures
            concentration_pred = concentration_prediction_norm * y_std + y_mean
            concentration_std = concentration_std_norm * y_std
            # concentration_mse = concentration_mse_norm * y_std**2

            results[contaminant][kernel_name] = {
                'prediction': concentration_pred,
                'std_dev': concentration_std,
                'log_likelihood': log_likelihood_norm
            }

    return results


In [ ]:
kernels= [
    ("RBF", GPy.kern.RBF(input_dim=2)),
    ("Matern32", GPy.kern.Matern32(input_dim=2)),
    ("Matern52", GPy.kern.Matern52(input_dim=2)),
    ("Linear", GPy.kern.Linear(input_dim=2)),
    ("RBF + Linear", GPy.kern.RBF(input_dim=2) + GPy.kern.Linear(input_dim=2))
]
contaminants = ["nox", "no2", "no", "o3"]
normalized_data = normalize_contaminants_data(df_air_q, contaminants)
results = predict_with_different_kernels(df_air_q, contaminants, kernels, normalized_data)


Processing contaminant: nox
  - Using kernel: RBF
Shape of y_train: (8, 1)


  - Using kernel: Matern32
Shape of y_train: (8, 1)


  - Using kernel: Matern52
Shape of y_train: (8, 1)


  - Using kernel: Linear
Shape of y_train: (8, 1)


  - Using kernel: RBF + Linear
Shape of y_train: (8, 1)


...


Processing contaminant: o3
  - Using kernel: RBF
Shape of y_train: (8, 1)


  - Using kernel: Matern32
Shape of y_train: (8, 1)


  - Using kernel: Matern52
Shape of y_train: (8, 1)


  - Using kernel: Linear
Shape of y_train: (8, 1)


  - Using kernel: RBF + Linear
Shape of y_train: (8, 1)


In [ ]:
def find_best_kernel(results):
    """
    Finds the best kernel for each contaminant based on the minimum Mean Squared Error (MSE).

    Args:
        results: Dictionary containing predictions, standard deviations, and MSE for each contaminant and kernel.

    Returns:
        best_kernels: Dictionary containing the best kernel for each contaminant, along with its MSE, prediction, and standard deviation.
    """
    best_kernels = {}

    for contaminant, contaminant_results in results.items():
        print(f"\nResultados para el contaminante: {contaminant}")

        best_kernel = None
        best_log_likelihood = -float('inf')

        for kernel_name, result in contaminant_results.items():
            log_likelihood = result['log_likelihood']
            prediction = result['prediction']
            std = result['std_dev']
            print(f"  - Kernel: {kernel_name}, Log-Likelihood: {log_likelihood:.4f}")

            if log_likelihood > best_log_likelihood:
                best_log_likelihood = log_likelihood
                best_kernel = kernel_name
                best_pred = prediction
                best_std = std

        best_kernels[contaminant] = (best_kernel, best_log_likelihood, best_pred, best_std)
        print(f"Mejor kernel para {contaminant}: {best_kernel} con Log-Likelihood: {best_log_likelihood:.4f}")

    return best_kernels

In [ ]:
best_kernels = find_best_kernel(results)

print("\nMejores kernels para cada contaminante:")
for contaminant, (kernel, log_likelihood, pred, std) in best_kernels.items():
    print(f"- {contaminant}: Mejor kernel = {kernel}, Log-Likelihood = {log_likelihood:.4f}")


Resultados para el contaminante: nox
  - Kernel: RBF, Log-Likelihood: -10.0681
  - Kernel: Matern32, Log-Likelihood: -10.6580
  - Kernel: Matern52, Log-Likelihood: -10.4762
  - Kernel: Linear, Log-Likelihood: -11.1607
  - Kernel: RBF + Linear, Log-Likelihood: -10.0681
Mejor kernel para nox: RBF con Log-Likelihood: -10.0681

Resultados para el contaminante: no2
  - Kernel: RBF, Log-Likelihood: -11.1425
  - Kernel: Matern32, Log-Likelihood: -11.1713
  - Kernel: Matern52, Log-Likelihood: -11.1657
  - Kernel: Linear, Log-Likelihood: -11.2368
  - Kernel: RBF + Linear, Log-Likelihood: -11.1425
Mejor kernel para no2: RBF + Linear con Log-Likelihood: -11.1425

Resultados para el contaminante: no
  - Kernel: RBF, Log-Likelihood: -10.1914
  - Kernel: Matern32, Log-Likelihood: -10.6657
  - Kernel: Matern52, Log-Likelihood: -10.5211
  - Kernel: Linear, Log-Likelihood: -11.3515
  - Kernel: RBF + Linear, Log-Likelihood: -10.1914
Mejor kernel para no: RBF con Log-Likelihood: -10.1914

Resultados par


The results show that the RBF kernel performs well across multiple contaminants, achieving the highest log-likelihood for nox, no, and o3.


For no2, however, the RBF + Linear combination provides a slight advantage over the RBF kernel alone. This might indicate that the no2 contaminant data has a linear trend component in addition to a non-linear pattern


This suggest that the spatial variations in these contaminants tend to have smooth, continuous patterns without abrupt changes.


We addressed the problem of hyperparameter learning by using the Gaussian Process (GP) regression models in GPy, which automatically learns the hyperparameters of the chosen kernels through an optimization process.

1. We defined several kernels and their combinations to model the air quality data. .

2. For each kernel, the GP model has hyperparameters, such as the lengthscale and variance of the kernel, and the noise variance.

3. We let the GPy library automatically optimize these hyperparameters by maximizing the log-likelihood of the model.


The hyperparameter learning was automated via GPy's optimization routine, which used the data to tune the kernel parameters. This optimization was driven by the goal of maximizing the log-likelihood, ensuring that the kernel hyperparameters were chosen in a way that provided the best fit to the training data.

## Visualize results

In [ ]:
for contaminant, (kernel, mse, pred, std) in best_kernels.items():
  fig = MapPlotter()

  # Adding the stations as points in the maps:
  fig.add_longlat_points(
      df_stations["longitude"],
      df_stations["latitude"],
      text=df_stations.station,
      color="white",
      edgecolor="black",
  )

  # Adding the predicted concentration as a heatmap in the grid
  fig.add_raster_heatmap(grid[0], grid[1], pred, opacity=0.6)

  fig.fig.update_layout(
        title=f"Predicted Concentration for {contaminant}",
        title_x=0.5,
        title_y=0.95,  # Adjusts the vertical position of the title
        margin={"r": 0, "t": 50, "l": 0, "b": 10},  # Increases top margin for title
    )

  # Show map
  fig.show()


The colored heatmap represents the predicted concentration of the contaminant in different areas of the city. The color scale on the right indicates the levels of concentration in units of µg/m³

nox: High concentrations are centered in the city’s core, specifically in the Casco Antiguo and Delicias areas.

no2: the highest concentration also in Casco Antiguo and Delicias. However, there's a slightly different distribution, with a notable reduction in concentration in certain peripheral areas

no: The concentration is highest again in the central zones, with broader spread compared to NOx and NO₂. The impact area is more diffused but still centered around Casco Antiguo and surrounding neighborhoods.

o3: higher concentrations appear on the peripheral zones, particularly around Picarral and La Almozara. This difference is expected, as ozone typically forms from photochemical reactions and can accumulate in areas farther from NOx emission sources, especially in less densely urbanized locations.


In [ ]:
for contaminant, (kernel, mse, pred, std) in best_kernels.items():
  fig = MapPlotter()

  # Adding the stations as points in the maps:
  fig.add_longlat_points(
      df_stations["longitude"],
      df_stations["latitude"],
      text=df_stations.station,
      color="white",
      edgecolor="black",
  )

  # Adding the predicted concentration as a heatmap in the grid
  fig.add_raster_heatmap(grid[0], grid[1], std, opacity=0.6)

  fig.fig.update_layout(
        title=f"Predicted Uncertainty (Std Dev) for {contaminant}",
        title_x=0.5,
        title_y=0.95,  # Adjusts the vertical position of the title
        margin={"r": 0, "t": 50, "l": 0, "b": 10},  # Increases top margin for title
    )

  # Show map
  fig.show()

These maps show the predicted uncertainty (standard deviation) in the concentration estimates for each contaminant across Zaragoza. Higher uncertainty areas may indicate regions where predictions are less certain, often due to a lack of nearby measurement stations or more variable data patterns.

NOx: Higher uncertainty is concentrated in the northeastern and southwestern outskirts of the city, especially near areas lacking measurement stations.
In contrast, central regions around measurement stations show lower uncertainty

NO2: Uncertainty is notably high in peripheral zones such as north and southwest areas. These locations are far from the measurement stations and reflect greater variability.


NO: high uncertainty appears in outlying areas with limited station coverage, while lower uncertainty is found in central and southern areas close to stations.

O3: Significantly high uncertainty is observed across much of the map, particularly in the north and southwest regions, as well as areas outside station coverage.
Lower uncertainty in the central urban core suggests better model confidence where stations are more concentrated.



### Choose station

If a new station is to be added, it would be best to place it in an area where the uncertainty (standard deviation) is highest, as shown in the redder regions of the uncertainty maps. This high uncertainty indicates areas with fewer data points, which leads to less accurate predictions. Therefore, an ideal location for a new monitoring station would be around the Montecanal area. Adding a station here would likely improve the model's predictions by providing additional data in an area where the model currently struggles with accuracy.